In [1]:
from __future__ import print_function, division
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
%matplotlib inline

import time

from halotools.sim_manager import CachedHaloCatalog, UserSuppliedHaloCatalog
#halocat = CachedHaloCatalog(simname='bolplanck', halo_finder='rockstar',
#                            redshift=0.0, version_name='halotools_v0p4')

from halotools.empirical_models import HodModelFactory
from halotools.empirical_models.ia_models.ia_model_components import CentralAlignment, RandomAlignment, RadialSatelliteAlignment, SubhaloAlignment
from halotools.empirical_models.ia_models.ia_strength_models import RadialSatelliteAlignmentStrength
from halotools.empirical_models import TrivialPhaseSpace, ZuMandelbaum15Cens, ZuMandelbaum15Sats, \
                                        Leauthaud11Cens, Leauthaud11Sats, Zheng07Cens, Zheng07Sats, \
                                        NFWPhaseSpace, SubhaloPhaseSpace
from halotools.mock_observables import tpcf
from halotools.mock_observables.ia_correlations import ee_3d, ed_3d

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Eliminate halos with 0 for halo_axisA_x(,y,z)
def mask_bad_halocat(halocat):
    bad_mask = (halocat.halo_table["halo_axisA_x"] == 0) & (halocat.halo_table["halo_axisA_y"] == 0) & (halocat.halo_table["halo_axisA_z"] == 0)
    bad_mask = bad_mask ^ np.ones(len(bad_mask), dtype=bool)
    halocat._halo_table = halocat.halo_table[ bad_mask ]

# Set Up Variables
Prepare the values and variables to be used during model creation

## Halocat
Select which halocat to use

In [3]:
use_bolplanck = True

if use_bolplanck:
    halocat = CachedHaloCatalog(simname='bolplanck', halo_finder='rockstar', redshift=0, version_name='halotools_v0p4')
    mask_bad_halocat(halocat)
else:
    halocat = CachedHaloCatalog(simname='multidark', halo_finder='rockstar', redshift=0, version_name='halotools_v0p4')
    mask_bad_halocat(halocat)

## Rbins
Choose the rbin values

In [4]:
rbins = np.logspace(-1,1.2,15)
rbin_centers = (rbins[:-1]+rbins[1:])/2.0

## Build Model
Set up model components.</br>
These may change throughout the run, especially the alignment strengths.</br>
To this end, I have placed the whole model generation into a function that can be called using just the alignment parameters.</br>
Other parameters may be added as flexibility is needed.

In [5]:
central_alignment_strength = 1
satellite_alignment_a=0.805
satellite_alignment_gamma=-0.029
sat_bins = np.logspace(10.5, 15.2, 15)
if not use_bolplanck:
    sat_bins = np.logspace(12.4, 15.5, 15)

In [19]:
def build_model_instance(cen_strength, sat_a, sat_gamma, sat_bins, halocat):

    cens_occ_model = Zheng07Cens()
    cens_prof_model = TrivialPhaseSpace()
    cens_orientation = CentralAlignment(central_alignment_strenth=cen_strength)

    sats_occ_model = Zheng07Sats()
    prof_args = ("satellites", sat_bins)
    sats_prof_model = SubhaloPhaseSpace(*prof_args)

    #sats_orientation = SubhaloAlignment
    sats_orientation = RadialSatelliteAlignment(satellite_alignment_strength=1, halocat=halocat)
    sats_strength = RadialSatelliteAlignmentStrength(satellite_alignment_a=sat_a, satellite_alignment_gamma=sat_gamma)
    Lbox = halocat.Lbox
    sats_strength.inherit_halocat_properties(Lbox=Lbox)
    
    model_instance = HodModelFactory(centrals_occupation = cens_occ_model,
                                     centrals_profile = cens_prof_model,
                                     satellites_occupation = sats_occ_model,
                                     satellites_profile = sats_prof_model,
                                     satellites_radial_alignment_strength = sats_strength,
                                     centrals_orientation = cens_orientation,
                                     satellites_orientation = sats_orientation,
                                     model_feature_calling_sequence = (
                                     'centrals_occupation',
                                     'centrals_profile',
                                     'satellites_occupation',
                                     'satellites_profile',
                                     'satellites_radial_alignment_strength',
                                     'centrals_orientation',
                                     'satellites_orientation')
                                    )

    seed=None
    model_instance.populate_mock(halocat,seed=seed)
    
    return model_instance

# Generate Data

In [7]:
def generate_correlations(model, halocat):
    gal_table = model.mock.galaxy_table

    coords = np.array( [ gal_table["x"], gal_table["y"], gal_table["z"] ] ).T
    orientations = np.array( [ gal_table["galaxy_axisA_x"], gal_table["galaxy_axisA_y"], gal_table["galaxy_axisA_z"] ] ).T
    
    xi = tpcf(coords, rbins, coords, period=halocat.Lbox)
    omega = ed_3d(coords, orientations, coords, rbins, period=halocat.Lbox)
    eta = ee_3d(coords, orientations, coords, orientations, rbins, period=halocat.Lbox)
    
    return [xi, omega, eta]

In [8]:
central_alignment_strengths = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
satellite_alignment_params = [(0.75, -0.02), (0.805, -0.029), (0.85, -0.35)]
runs = 1

# Initial model (Using default values from above)
model_instance = build_model_instance(central_alignment_strength, satellite_alignment_a, satellite_alignment_gamma, sat_bins, halocat)

input_params = []
outputs = []

start = time.time()

for cen_mu in central_alignment_strengths:
    for sat_a, sat_gamma in satellite_alignment_params:
        for run in range(runs):
            input_row = [cen_mu, sat_a, sat_gamma]
            
            # Adjust parameters in model
            model_instance.param_dict["central_alignment_strength"] = cen_mu
            model_instance.param_dict["a"] = sat_a
            model_instance.param_dict["gamma"] = sat_gamma
            model_instance.mock.populate()
            
            output_row = generate_correlations(model_instance, halocat)
            
            input_params.append( input_row )
            outputs.append( output_row )
            
input_params = np.array( input_params )
outputs = np.array( outputs )

In [ ]:
np.save("input_params.npy", input_params)
np.save("outputs.npy", outputs)

# Testing Ground

In [20]:
model_instance = build_model_instance(central_alignment_strength, satellite_alignment_a, satellite_alignment_gamma, sat_bins, halocat)

In [21]:
model_instance.param_dict

{'logMmin': 12.02,
 'sigma_logM': 0.26,
 'logM0': 11.38,
 'logM1': 13.31,
 'alpha': 1.06,
 'a': 0.805,
 'gamma': -0.029,
 'central_alignment_strength': 1.0,
 'satellite_alignment_strength': 1}

In [28]:
param_dict = {
    "central_alignment_strength":[-1.,-0.5,0,0.5,1.],
    "satellite_alignment_strength":[-1.,-0.5,0,0.5,1.],
    "logMmin":[12.0, 12.5, 13.0],
    "sigma_logM":[0.22, 0.26, 0.3],
    "logM0":[12.3, 12.6, 12.9],
    "logM1":[13.2, 13.5, 13.8],
    "alpha":[0.5, 1.0, 1.5]
}

In [23]:
def permute_params(param_dict):
    keys = [ key for key in param_dict.keys() ]
    values = list( itertools.product( *[ param_dict[key] for key in keys ] ) )

    return keys, values

In [24]:
import itertools

In [29]:
keys, values = permute_params(param_dict)

In [31]:
len(values)/200

30.375